In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from arch import arch_model
import pandas as pd
from pandas.tseries.offsets import MonthEnd, YearEnd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

### Returns Daily

In [3]:
daily_returns = pd.read_csv('Final_Returns_Data.csv', index_col=0)
daily_returns.index = pd.to_datetime(daily_returns.index)

### Returns Monthly

In [4]:
notional = pd.read_csv('Final_Notional_Data.csv', index_col=0, parse_dates=True)
# Step 1: Create a DataFrame of month-ends
month_ends = pd.date_range(start=notional.index.min(), end=notional.index.max(), freq='M')

# Step 2: For each month-end, find the closest date in df.index
closest_dates = []
for month_end in month_ends:
    closest_date = notional.index[np.abs((notional.index - month_end).days).argmin()]
    closest_dates.append(closest_date)

# Step 3: Extract rows at those closest dates
df_closest_to_month_end = notional.loc[sorted(set(closest_dates))]

returns_monthly = df_closest_to_month_end.pct_change().dropna()

# Deal with Athens
# Simple Linear interpolation
idx = returns_monthly["FTSE/Athens Notional Value"].idxmax()
pos = returns_monthly.index.get_loc(idx)
prior_date = returns_monthly.index[pos - 1]
returns_monthly.loc[prior_date]
following_date = returns_monthly.index[pos + 1]

# Add the average of the two surrounding values
returns_monthly["FTSE/Athens Notional Value"].loc[returns_monthly["FTSE/Athens Notional Value"].idxmax()] = (returns_monthly["FTSE/Athens Notional Value"].loc[following_date] + returns_monthly["FTSE/Athens Notional Value"].loc[prior_date]) / 2

returns_monthly.describe()

,DAX Notional Value,Amsterdam Index Notional Value,CAC40 Notional Value,PSI-20 Notional Value,IBEX-35 Notional Value,FTSE/Athens Notional Value,BEL-20 Notional Value,FTSE/MIB Notional Value,Gold Notional Value,Silver Notional Value
count,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000
mean,0.007121,0.006591,0.005938,0.002680,0.005943,0.001997,0.005134,0.005139,0.009129,0.010669
std,0.051665,0.047431,0.047944,0.050424,0.054821,0.091698,0.048334,0.058607,0.047736,0.092569
min,-0.173032,-0.199692,-0.168957,-0.202864,-0.215921,-0.305530,-0.208714,-0.217006,-0.180050,-0.279791
25%,-0.023088,-0.019074,-0.023549,-0.024552,-0.022831,-0.047848,-0.020714,-0.027160,-0.020996,-0.054097
50%,0.013294,0.012669,0.009803,0.005193,0.007164,0.008565,0.013739,0.009324,0.005240,0.000986
75%,0.037859,0.035262,0.036580,0.030809,0.035203,0.058153,0.034870,0.040573,0.039517,0.066292
max,0.169795,0.138109,0.202521,0.167766,0.258393,0.315495,0.210823,0.232054,0.140449,0.304222


In [5]:
returns_monthly.index = returns_monthly.index.to_period('M')
returns_monthly

,DAX Notional Value,Amsterdam Index Notional Value,CAC40 Notional Value,PSI-20 Notional Value,IBEX-35 Notional Value,FTSE/Athens Notional Value,BEL-20 Notional Value,FTSE/MIB Notional Value,Gold Notional Value,Silver Notional Value
Date,,,,,,,,,,
2004-05,-0.011564,-0.003418,0.012284,-0.039936,-0.013116,-0.027526,0.011813,-0.003884,0.017823,0.004100
2004-06,0.033241,0.021238,0.021609,0.028331,0.012084,-0.003642,0.033888,0.038000,-0.003806,-0.054212
2004-07,-0.044776,-0.043450,-0.024490,-0.036205,-0.014027,-0.015930,-0.002984,-0.020271,-0.004075,0.132930
2004-08,-0.027920,-0.011933,-0.014759,-0.006005,-0.004217,-0.007472,0.031859,-0.024127,0.049356,0.033269
2004-09,0.024865,-0.000079,0.009567,0.035819,0.019023,0.023637,0.051451,0.030095,0.019976,0.020465
...,...,...,...,...,...,...,...,...,...,...
2024-11,0.026995,0.008779,-0.018203,-0.019817,-0.000465,0.009610,0.002015,-0.017490,-0.030706,-0.061212
2024-12,0.009854,-0.007066,0.017121,-0.010945,-0.005875,0.062043,0.008392,0.019002,-0.011519,-0.057780
2025-01,0.087971,0.045883,0.075290,0.022646,0.066448,0.062129,0.012585,0.067697,0.069101,0.109015


In [6]:
idx = daily_returns["FTSE/Athens Notional Return"].idxmax()
position = daily_returns.index.get_loc(idx)
prior_day_retun = daily_returns["FTSE/Athens Notional Return"].iloc[position-1]
following_day_return = daily_returns["FTSE/Athens Notional Return"].iloc[position+1]
daily_returns["FTSE/Athens Notional Return"].iloc[position] = (prior_day_retun + following_day_return) / 2

In [7]:
daily_returns_copy = daily_returns.copy()
daily_returns_copy.index = daily_returns_copy.index.to_period('M')

############################################################################################
# Simple Scaling Approach
############################################################################################
# Monthy Realized Variance
monthly_var = daily_returns_copy.groupby(daily_returns_copy.index).var() * 21 # daily to monthly var
monthly_var.columns = returns_monthly.columns

# Lag monthly_var by 1 period to use previous month's realized variance
monthly_var_lagged = monthly_var.shift(1)
monthly_var_lagged.columns = returns_monthly.columns
returns_monthly_scaled = returns_monthly.divide(monthly_var_lagged.loc[returns_monthly.index])
returns_monthly_scaled

############################################################################################
#Semi-variance Approach
############################################################################################
# Calculate the semi-variance
def semi_variance(returns):
    # Calculate the mean of the returns
    mean_return = returns.mean()
    # Calculate the semi-variance
    return ((returns[returns < mean_return] - mean_return) ** 2).mean()

semi_var_monthly = daily_returns_copy.groupby(daily_returns_copy.index).apply(semi_variance) * 21 # daily to monthly var
semi_var_monthly.columns = returns_monthly.columns
semi_var_monthly_lagged = semi_var_monthly.shift(1)
returns_monthly_scaled_semi = returns_monthly.divide(semi_var_monthly_lagged.loc[returns_monthly.index])

############################################################################################
# GARCH(1,1) Model
############################################################################################

# Initialize result container
rolling_garch_vol = pd.DataFrame(index=daily_returns.index, columns=daily_returns.columns)

# Loop over assets
for column in daily_returns.columns:
    returns_series = daily_returns[column].dropna()
    n = len(returns_series)
    
    # So! Here we are trying to loop over non-overlapping (important!) windows of 21 days
    for i in range(0, n // 21):
        start = i * 21
        end = (i + 1) * 21

        window_data = returns_series.iloc[start:end]

        if len(window_data) < 21:
            continue  # Not enough data

        # Let's fit GARCH(1,1) model here
        try:
            model = arch_model(window_data, vol='Garch', p=1, q=1)
            fitted_model = model.fit(disp='off')

            # Forecast 21 steps ahead (not just one, but model recursively forecasts 21 steps ahead)
            forecast = fitted_model.forecast(horizon=21)

            # Extract variances
            forecasted_variance = forecast.variance.values[-1, :]

            # Assign to corresponding index in DataFrame
            idx = returns_series.index[start:end]
            rolling_garch_vol.loc[idx, column] = forecasted_variance[:len(idx)]
        
        except Exception as e:
            print(f"Error for {column}, window {i}: {e}")

# All acrobatics to convert vol daily to monthly, I also messed up column names so names are adjusted manualy
rolling_garch_vol = rolling_garch_vol.dropna()
rolling_garch_vol.index = rolling_garch_vol.index.to_period('M')
rolling_garch_vol_monthly = rolling_garch_vol.groupby(rolling_garch_vol.index).sum()
rolling_garch_vol_monthly.columns = returns_monthly.columns
rolling_garch_vol_monthly_lagged = rolling_garch_vol_monthly.shift(1)
returns_monthly_scaled_garch = returns_monthly.divide(rolling_garch_vol_monthly_lagged .loc[returns_monthly.index])

/opt/anaconda3/lib/python3.12/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(

In princile what I did with the GARCH model is to use the GARCH(1,1) model to forecast the volatility of the returns for non-overlapping 21 days windows. We use the window to recursively forecast daily variance 21 days ahead using fitted out of sample data.

It might seem weird to forecst on non-rolling windows because we are not using incoming data, but let's just assume that for the sake of lower computational complexity. We have to forecast it somehow and this is only approach were out of sample approach is feasibe (instead in sample approach used by acadmics)

###  How does recursive GARCH(1,1) forecasting work?

The GARCH(1,1) model estimates conditional variance at time $t$ as:

$$
\sigma_t^2 = \omega + \alpha \cdot \epsilon_{t-1}^2 + \beta \cdot \sigma_{t-1}^2
$$

Where:
- $\omega > 0$ is a constant,
- $\alpha \geq 0$ captures the impact of past shocks (ARCH term),
- $\beta \geq 0$ captures the persistence in volatility (GARCH term),
- $\epsilon_{t-1}$ is the return shock at time $t-1$ (residual),
- $\sigma_{t-1}^2$ is the forecast variance from the previous day.

To **recursively forecast** volatility $h$ steps ahead (e.g., 21 days), we use the following logic:

1. At step $t+1$, we plug in the last observed $\epsilon_t^2$ and $\sigma_t^2$.
2. For $t+2$, we assume that the expected future shock $\mathbb{E}[\epsilon_{t+1}^2] = \sigma_{t+1}^2$ and continue:

\[
\begin{align*}
\sigma_{t+1}^2 &= \omega + \alpha \cdot \epsilon_{t}^2 + \beta \cdot \sigma_{t}^2 \\
\sigma_{t+2}^2 &= \omega + (\alpha + \beta) \cdot \sigma_{t+1}^2 \\
\sigma_{t+3}^2 &= \omega + (\alpha + \beta) \cdot \sigma_{t+2}^2 \\
&\vdots \\
\sigma_{t+h}^2 &= \omega + (\alpha + \beta) \cdot \sigma_{t+h-1}^2
\end{align*}
\]

This recursion converges to the long-term variance:

$$
\sigma_{\infty}^2 = \frac{\omega}{1 - \alpha - \beta}
$$

---


In [8]:
#Normal Approach
returns_monthly_scaled

,DAX Notional Value,Amsterdam Index Notional Value,CAC40 Notional Value,PSI-20 Notional Value,IBEX-35 Notional Value,FTSE/Athens Notional Value,BEL-20 Notional Value,FTSE/MIB Notional Value,Gold Notional Value,Silver Notional Value
Date,,,,,,,,,,
2004-05,-3.546244,-2.500370,6.648708,-1209.974371,-10.882048,-11.280210,14.643370,-5.391877,4.117981,0.176677
2004-06,8.384770,6.301023,6.916753,16.557058,3.907033,-0.931034,14.127456,17.230419,-1.598296,-4.502137
2004-07,-20.525909,-39.756182,-18.927971,-84.844216,-10.085494,-9.797557,-5.030667,-34.821746,-1.932077,13.366909
2004-08,-14.589171,-6.821080,-12.544668,-6.920721,-4.493462,-6.455176,24.780990,-36.044876,23.727712,3.908407
2004-09,13.021140,-0.036144,5.409797,94.167828,17.393260,11.081917,48.906202,30.303025,11.231576,5.866395
...,...,...,...,...,...,...,...,...,...,...
2024-11,33.271095,6.572871,-16.735667,-15.660340,-0.428268,6.013041,3.141568,-14.041239,-22.642442,-6.435887
2024-12,5.067072,-5.003864,9.577625,-5.558110,-2.900831,43.068923,4.362168,9.717747,-2.970586,-9.759971
2025-01,132.892845,75.917777,85.993467,48.405379,46.801529,51.152995,21.319222,59.111480,37.742274,16.150648


In [9]:
# Semi-variance scaling
returns_monthly_scaled_semi

,DAX Notional Value,Amsterdam Index Notional Value,CAC40 Notional Value,PSI-20 Notional Value,IBEX-35 Notional Value,FTSE/Athens Notional Value,BEL-20 Notional Value,FTSE/MIB Notional Value,Gold Notional Value,Silver Notional Value
Date,,,,,,,,,,
2004-05,-2.965227,-2.041017,6.825137,-1022.611652,-9.298138,-12.458461,24.940773,-7.649870,0.817370,0.154669
2004-06,6.055804,4.816990,7.145116,13.521150,3.715025,-0.816106,8.865505,17.085585,-1.655731,-3.462063
2004-07,-27.381165,-23.905962,-17.359234,-86.480621,-10.148159,-10.695722,-5.162692,-36.937633,-1.872278,16.662432
2004-08,-31.382064,-7.093673,-12.860275,-6.908392,-3.634919,-6.469822,45.051638,-38.760936,31.060772,4.461639
2004-09,12.104559,-0.031772,4.274248,63.985089,14.712075,9.719084,43.272453,23.571015,11.034037,3.338212
...,...,...,...,...,...,...,...,...,...,...
2024-11,38.352694,5.150142,-17.014290,-40.411393,-0.376978,6.347791,3.400333,-14.327378,-14.771634,-5.099913
2024-12,5.246998,-5.588489,7.963561,-4.024090,-1.977997,43.946376,2.846999,11.951928,-1.521889,-7.204954
2025-01,182.824821,56.174961,74.987103,37.094776,35.764777,128.302671,12.261375,66.581187,38.336011,13.378645


In [10]:
# GARCH(1,1) scaling
returns_monthly_scaled_garch

,DAX Notional Value,Amsterdam Index Notional Value,CAC40 Notional Value,PSI-20 Notional Value,IBEX-35 Notional Value,FTSE/Athens Notional Value,BEL-20 Notional Value,FTSE/MIB Notional Value,Gold Notional Value,Silver Notional Value
Date,,,,,,,,,,
2004-05,-6.145174,-3.148479,12.044258,-70.81575,-8.971419,-23.385839,10.829113,-5.766694,18.731751,0.735971
2004-06,6.955817,8.501519,8.751896,23.239072,3.112107,-1.252621,12.160728,24.260507,-1.473114,-3.975227
2004-07,-27.97399,-43.145091,-22.650895,-56.430759,-13.96988,-11.140569,-4.772909,-36.335333,-1.702032,14.613524
2004-08,-14.857357,-6.483258,-11.053316,-8.834706,-4.422369,-5.1501,32.079869,-30.690024,23.327439,3.93026
2004-09,12.972066,-0.03822,5.697505,74.247049,17.113698,11.714458,40.878062,29.65036,13.101828,3.833855
...,...,...,...,...,...,...,...,...,...,...
2024-11,31.757858,4.318382,-15.438227,-25.482108,-0.494604,5.729128,2.650182,-13.221794,-35.835786,-6.705164
2024-12,5.503295,-5.279279,11.429036,-10.897519,-3.086215,49.602208,4.384512,10.601775,-3.060339,-16.952204
2025-01,110.361936,80.999126,82.057116,43.308116,46.913422,99.189043,23.898361,58.266488,39.443152,15.150933
